In [12]:
import pandas as pd
dados = pd.read_csv('dados_unidade_3.csv')
dados.columns = [x.lower() for x in dados.columns]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
escalonar = dados.iloc[:,:10]
escalonar = pd.DataFrame(scaler.fit_transform(escalonar),columns=escalonar.columns)
escalonar['nox'] = dados['nox'].values
x = escalonar.iloc[:,:10].values
y = escalonar.iloc[:,10].values
from sklearn.model_selection import train_test_split
x_treino,x_teste,y_treino,y_teste = train_test_split(x,y,train_size=0.8,random_state=3)

In [13]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,mean_absolute_percentage_error
import math as mt
results = pd.DataFrame(columns = ['modelo','r2','mae','mse','rmse','mape'])
def add_result(results,nome,y_teste,y_prev):
    #uso: results = add_results(...)
    data = [
        nome,
        r2_score(y_teste,y_prev),
        mean_absolute_error(y_teste,y_prev),
        mean_squared_error(y_teste,y_prev),
        mt.sqrt(mean_squared_error(y_teste,y_prev)),
        mean_absolute_percentage_error(y_teste,y_prev)
    ]
    results.loc[results.shape[0]]=data
    return results

In [14]:
from sklearn.tree import DecisionTreeRegressor
nome = 'Árvore de decisao - dados padrão'
modelo = DecisionTreeRegressor()
modelo.fit(x_treino,y_treino)
y_prev = modelo.predict(x_teste)
results = add_result(results,nome,y_teste,y_prev)
results

,modelo,r2,mae,mse,rmse,mape
0,Árvore de decisao - dados padrão,0.756617,3.413182,32.576372,5.707571,0.052483


# Redução de dimensionalidade

Técnica utilizada para simplificar dados de alta dimensão, transformando-os em um formato de menor dimensão, mantendo ao máximo a informação relevante

## PCA - Principal component analysis

O PCA transforma as variáveis originais em um novo conjunto de variáveis chamadas componentes principais, que são ortogonais (perpendiculares) entre si e refletem a máxima variância dos dados.

In [15]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)  
x_pca = pca.fit_transform(x_treino)

In [16]:
pd.DataFrame(x_pca)

,0,1
0,0.123921,0.154009
1,0.644239,-0.149964
2,-0.037584,-0.067556
3,-0.645502,-0.165761
4,0.710392,-0.309173
...,...,...
29381,-0.602286,-0.125672
29382,-0.060018,0.213212
29383,0.132578,0.471035
29384,-0.023038,-0.153344


In [17]:
nome = 'Árvore de decisao - pca de 2 componentes'
x_teste_pca = pca.transform(x_teste)
modelo = DecisionTreeRegressor()
modelo.fit(x_pca,y_treino)
y_prev = modelo.predict(x_teste_pca)
results = add_result(results,nome,y_teste,y_prev)
results

,modelo,r2,mae,mse,rmse,mape
0,Árvore de decisao - dados padrão,0.756617,3.413182,32.576372,5.707571,0.052483
1,Árvore de decisao - pca de 2 componentes,-0.057521,8.809818,141.547279,11.897364,0.135658


Automatizar o pca

In [18]:
x_teste.shape

(7347, 10)

In [19]:
for i in range(3,10):
    pca = PCA(n_components=i)  
    x_pca = pca.fit_transform(x_treino)
    nome = f'Árvore de decisao - pca de {i} componentes'
    print(nome)
    x_teste_pca = pca.transform(x_teste)
    modelo = DecisionTreeRegressor()
    modelo.fit(x_pca,y_treino)
    y_prev = modelo.predict(x_teste_pca)
    results = add_result(results,nome,y_teste,y_prev)
results

Árvore de decisao - pca de 3 componentes
Árvore de decisao - pca de 4 componentes
Árvore de decisao - pca de 5 componentes
Árvore de decisao - pca de 6 componentes
Árvore de decisao - pca de 7 componentes
Árvore de decisao - pca de 8 componentes
Árvore de decisao - pca de 9 componentes


,modelo,r2,mae,mse,rmse,mape
0,Árvore de decisao - dados padrão,0.756617,3.413182,32.576372,5.707571,0.052483
1,Árvore de decisao - pca de 2 componentes,-0.057521,8.809818,141.547279,11.897364,0.135658
2,Árvore de decisao - pca de 3 componentes,0.189576,7.474506,108.473775,10.415074,0.114887
3,Árvore de decisao - pca de 4 componentes,0.335939,6.398492,88.883390,9.427799,0.098234
4,Árvore de decisao - pca de 5 componentes,0.479165,5.367744,69.712815,8.349420,0.082045
5,Árvore de decisao - pca de 6 componentes,0.636975,4.223569,48.590205,6.970667,0.064214
6,Árvore de decisao - pca de 7 componentes,0.697431,3.705208,40.498328,6.363830,0.056220
7,Árvore de decisao - pca de 8 componentes,0.770107,3.277266,30.770789,5.547142,0.049680
8,Árvore de decisao - pca de 9 componentes,0.775163,3.297691,30.093982,5.485798,0.050248
